In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
LR = 1e-3
env = gym.make('CartPole-v1')

env.reset()

goal_steps = 500
score_requirement = 50
initial_games = 10000

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
def some_random_games_first():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
#             env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                break            

In [10]:
some_random_games_first()

In [4]:
def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            
            if len(prev_observation) > 0:
                game_memory.append([prev_observation, action ])
            prev_observation = observation
            score += reward
            if done:
                break
                
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                elif data[1] == 0:
                    output = [1, 0]
                    
                training_data.append([data[0], output])
                
        env.reset()
        scores.append(score)
    training_data_save = np.array(training_data)
    np.save('saved.npy', training_data_save)
    print('Average accepted score', mean(accepted_scores))
    print('Median accepted score', median(accepted_scores))
    print(Counter(accepted_scores))
    return training_data

In [13]:
initial_population()

Average accepted score 61.739837398373986
Median accepted score 58.0
Counter({52.0: 29, 50.0: 27, 57.0: 26, 56.0: 22, 51.0: 22, 62.0: 17, 58.0: 17, 53.0: 16, 55.0: 15, 63.0: 14, 67.0: 13, 60.0: 12, 54.0: 11, 70.0: 11, 66.0: 10, 59.0: 9, 61.0: 9, 74.0: 8, 68.0: 7, 65.0: 7, 64.0: 7, 73.0: 7, 71.0: 6, 72.0: 6, 82.0: 5, 69.0: 5, 75.0: 4, 89.0: 3, 90.0: 3, 78.0: 2, 77.0: 2, 81.0: 2, 79.0: 2, 76.0: 1, 84.0: 1, 104.0: 1, 91.0: 1, 87.0: 1, 102.0: 1, 86.0: 1, 97.0: 1, 95.0: 1, 94.0: 1, 105.0: 1, 88.0: 1, 127.0: 1})


[[array([-0.04909301,  0.23387057,  0.04091685, -0.23782511]), [0, 1]],
 [array([-0.0444156 ,  0.42838481,  0.03616035, -0.51732607]), [1, 0]],
 [array([-0.0358479 ,  0.23277286,  0.02581383, -0.21347114]), [0, 1]],
 [array([-0.03119245,  0.42751643,  0.02154441, -0.49790073]), [0, 1]],
 [array([-0.02264212,  0.62232809,  0.01158639, -0.78371685]), [1, 0]],
 [array([-0.01019555,  0.42704885, -0.00408794, -0.48741132]), [1, 0]],
 [array([-0.00165458,  0.23198481, -0.01383617, -0.19601957]), [1, 0]],
 [array([ 0.00298512,  0.03706348, -0.01775656,  0.09226674]), [0, 1]],
 [array([ 0.00372639,  0.23243538, -0.01591123, -0.20596506]), [0, 1]],
 [array([ 0.0083751 ,  0.4277812 , -0.02003053, -0.50362441]), [0, 1]],
 [array([ 0.01693072,  0.62317965, -0.03010302, -0.80255187]), [0, 1]],
 [array([ 0.02939431,  0.81870119, -0.04615405, -1.10455026]), [1, 0]],
 [array([ 0.04576834,  0.62421559, -0.06824506, -0.826697  ]), [1, 0]],
 [array([ 0.05825265,  0.43008993, -0.084779  , -0.5562348 ]), [

In [5]:
def neural_network_model(input_size):
    network = input_data(shape = [None, input_size, 1], name = 'input')
    
    network = fully_connected(network, 128, activation = 'relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation = 'relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 512, activation = 'relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation = 'relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 128, activation = 'relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 2, activation = 'softmax')
    network = regression(network, optimizer = 'adam', learning_rate = LR, loss = 'categorical_crossentropy', name = 'targets')
    
    model = tflearn.DNN(network, tensorboard_dir = 'log')
    
    return model

In [6]:
def train_model(training_data, model = False):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]), 1)
    y = [i[1] for i in training_data]
    if not model:
        model = neural_network_model (input_size=len(X[0]))
    model.fit({'input':X}, {'targets': y}, n_epoch = 6, snapshot_step = 500, show_metric = True, run_id = 'openaistuff')
    return model

In [7]:
training_data = initial_population()
model = train_model(training_data)

Training Step: 2027  | total loss: 0.66724 | time: 5.216s
| Adam | epoch: 006 | loss: 0.66724 - acc: 0.6004 -- iter: 21568/21593
Training Step: 2028  | total loss: 0.66504 | time: 5.234s
| Adam | epoch: 006 | loss: 0.66504 - acc: 0.6123 -- iter: 21593/21593
--


In [7]:
# model.save("First_model.model")
model.load('First_model.model')

NameError: name 'model' is not defined

In [8]:
scores = []
choices = []
for each_game in range(10):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        env.render()
        if len(prev_obs) == 0:
            action = random.randrange(0, 2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1, len(training_data[0][0]), 1))[0])
        choices.append(action)
        
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score += reward
        if done:
            break
scores.append(score)

print('Average score', sum(scores)/len(scores))
print('Choice 1 : {}, Choice 2 : {}'.format(choices.count(1)/len(choices), choices.count(0)/len(choices)))

Average score 500.0
Choice 1 : 0.4998, Choice 2 : 0.5002
